<a href="https://colab.research.google.com/github/jmsmg/TIL/blob/main/Python/Futures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Futures(동시성)
  > parallel tasks

- 비동기 작업 처리
- 파이썬 GIL 설명
- 동시성 처리 실습 예제
- Process, Thread 예제



In [ ]:
from concurrent.futures.process import ProcessPoolExecutor
# Futures 동시성
# 비동기 작업 실행
# 지연시간(Block) CPU 및 리소스 낭비 방지 -> (File)Network I/O 관련 작업 -> 동시성 활용 권장
# 비동기 작업과 적합한 프로그램일 경우 압도적으로 성능 향상

# futures : 비동기 실행을 위한 API를 고수준으로 작성 -> 사용하기 쉽도록 개선
# concurrent.futures
# 1. 멀티 스레딩/멀티프로세싱 API 통일 -> 매우 사용하기 쉬움
# 2. 실행중인 작업 취소, 완료 여부 체크, 타임아웃 옵션, 콜백 추가, 동기화 코드 쉽게 작성 -> Promise 개념

# 2가지 패턴 실습
# concurrent.futures 사용법1
# concurrent.futures 사용법2

# GIL (파이썬에만 있음) : 두개 이상의 스레드가 동시에 실행 될 때 하나의 자원을 액세스 하는 경우 -> 문제점을 방지하기 위해
#                         GIL이 실행 -> 리소스 전체에 락이 걸림 -> Context switch(문맥 교환비용)

# GIL : 멀티프로세싱 사용, Cpython

import os
import time
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, wait, as_completed
# import threading
# import multiprocessing

WORK_LIST = [100000, 1000000, 10000000, 100000000]


# 동시성 합계 메인 함수
# 누적 합계 함수(제너레이터)

def sum_generator(n):
  return sum(n for n in range(n, n+1))

def main():
  # Worker Count
  worker = min(10, len(WORK_LIST))

  # 시작 시간
  start_tm = time.time()

  # 결과 건수
  # ProcessPoolExcutor
  with ThreadPoolExecutor() as excutor:
    # map -> 작업 순서 유지, 즉시 실행 
    result = excutor.map(sum_generator, WORK_LIST)

  # 종료 시간
  end_tm = time.time() - start_tm

  # 출력 포맷
  msg = '\n Result -> {} Time : {:.2f}s'
  
  # 최종 결과 출력
  print(msg.format(list(result), end_tm))

# 실행
if __name__ == '__main__': # 시작점 코드
  main()


 Result -> [100000, 1000000, 10000000, 100000000] Time : 0.00s


In [3]:
import os
import time
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, wait, as_completed
# import threading
# import multiprocessing

WORK_LIST = [100000, 1000000, 10000000, 100000000]

def sum_generator(n):
  return sum(n for n in range(1, n+1))

# wait 
# as_completed

def main():
  # Worker Count
  worker = min(10, len(WORK_LIST))

  # 시작 시간
  start_tm = time.time()

  futures_list = []

  # 결과 건수
  # ProcessPoolExecutor
  with ProcessPoolExecutor() as excutor:
    for work in WORK_LIST:
      # future 반환
      future = excutor.submit(sum_generator, work)
      # 스케줄링
      futures_list.append(future)
      # 스케줄링 확인
      print(f'Scheduled for {work} : {future}')
      print()
    

    # # wait 결과 출력
    # result = wait(futures_list, timeout=7)
    # # 성공
    # print(f'Completed Tasks : {result.done}')
    # # 실패
    # print(f'Pending ones after waiting for 7seconds : {result.not_done}')
    # # 결과값 출력
    # print([future.result() for future in result.done])


    # as_completed 결과 출력
    for future in as_completed(futures_list):
      result = future.result()
      done = future.done()
      cancelled = future.cancelled

      # future 결과 확인
      print(f'Future Result : {result}, Done : {done}')
      print(f'Future cancelled : {cancelled}')


  # 종료 시간
  end_tm = time.time() - start_tm

  # 출력 포맷
  msg = '\n Time : {:.2f}s'
  
  # 최종 결과 출력
  print(msg.format(end_tm))

# 실행
if __name__ == '__main__': # 시작점 코드
  main()

Scheduled for 1000000 : <Future at 0x7f9ee0ae4190 state=running>

Scheduled for 10000000 : <Future at 0x7f9ee0ae4890 state=pending>

Scheduled for 100000000 : <Future at 0x7f9ee0ae4ad0 state=pending>

Scheduled for 1000000000 : <Future at 0x7f9ee0ae4d10 state=pending>

Future Result : 500000500000, Done : True
Future cancelled : <bound method Future.cancelled of <Future at 0x7f9ee0ae4190 state=finished returned int>>
Future Result : 50000005000000, Done : True
Future cancelled : <bound method Future.cancelled of <Future at 0x7f9ee0ae4890 state=finished returned int>>
Future Result : 5000000050000000, Done : True
Future cancelled : <bound method Future.cancelled of <Future at 0x7f9ee0ae4ad0 state=finished returned int>>
Future Result : 500000000500000000, Done : True
Future cancelled : <bound method Future.cancelled of <Future at 0x7f9ee0ae4d10 state=finished returned int>>

 Time : 73.44s
